In [8]:
import numpy as np
import pandas as pd
import keras
import warnings
warnings.filterwarnings(action='ignore')
import sys

from keras.models import Sequential
from keras.layers import Dense,Dropout

### Data 불러오기

In [9]:
data = pd.read_csv('data_preprocessing.csv')
data

,year,month,CARD_SIDO_NM,CARD_CCG_NM,STD_CLSS_NM,SEX,region_diff,AGE,CSTMR_CNT,CNT,AMT
0,2019,1,강원,강릉시,건강보조식품 소매업,1,1,20,4,4,311200
1,2019,1,강원,강릉시,건강보조식품 소매업,1,1,30,7,8,1374500
2,2019,1,강원,강릉시,건강보조식품 소매업,1,1,40,7,8,2764300
3,2019,1,강원,강릉시,건강보조식품 소매업,1,1,50,21,21,3690830
4,2019,1,강원,강릉시,건강보조식품 소매업,1,1,60,19,20,1433500
...,...,...,...,...,...,...,...,...,...,...,...
2691680,2020,3,충북,충주시,휴양콘도 운영업,1,1,30,3,4,43300
2691681,2020,3,충북,충주시,휴양콘도 운영업,1,1,40,3,3,35000
2691682,2020,3,충북,충주시,휴양콘도 운영업,1,1,50,4,6,188000
2691683,2020,3,충북,충주시,휴양콘도 운영업,1,1,60,3,3,194000


### SET Pred_Dataframe

In [10]:
def Pred_data_def(raw_data):
    temp         = []
    SEX          = raw_data['SEX'].unique()
    Region_diff  = raw_data['region_diff'].unique()
    AGE          = raw_data['AGE'].unique()
    month        = [4,7]
    CARD_SIDO_NM = raw_data['CARD_SIDO_NM'].unique()
    STD_CLSS_NM  = raw_data['STD_CLSS_NM'].unique()
    
    for sido in CARD_SIDO_NM:
        df = raw_data[raw_data["CARD_SIDO_NM"] == sido]
        ccg_list = df['CARD_CCG_NM'].unique()
        for ccg in ccg_list:
            for std in STD_CLSS_NM:
                for age in AGE: 
                    for sex in SEX:
                        for region in Region_diff:
                            for i in month:
                                temp.append([i, sido, ccg, std, sex, region, age])

    temp = np.array(temp)
    Pred_data = pd.DataFrame(data= temp, columns= ['month', 'CARD_SIDO_NM', 'CARD_CCG_NM',
                                                   'STD_CLSS_NM', 'SEX', 'region_diff', 'AGE'])
    return Pred_data

### Train, Pred 구분

In [11]:
Train_data = data.loc[(data['year']==2020)&((data['month']==1)|(data['month']==2)|
                                              (data['month']==3))]
Train_data_X = Train_data.drop(['AMT','year','CNT','CSTMR_CNT'],axis=1)
Train_data = Train_data.drop(['CNT','year','CSTMR_CNT'],axis=1)
Pred_data = Pred_data_def(Train_data_X)

print('Pred_data :',Pred_data.shape,
     '\nTrain_data :',Train_data.shape,
     '\nTrain_data_X :',Train_data_X.shape)

Train_data_en = pd.get_dummies(Train_data, columns=dum_col)
Pred_data_en  = pd.get_dummies(Pred_data, columns=dum_col)

Pred_data : (861000, 7) 
Train_data : (521750, 8) 
Train_data_X : (521750, 7)


### Model

In [ ]:
### 세팅중
# pred_data   = Pred_data.copy()
# pred_data.drop(['SEX','region_diff','AGE','CARD_SIDO_NM','STD_CLSS_NM'],axis=1, inplace=True)
final_data  = []
sido_list   = Train_data_en['CARD_SIDO_NM'].unique()[1:2]
std_list    = Train_data_en['STD_CLSS_NM'].unique()
dum_col     = ['CARD_CCG_NM','SEX','region_diff','AGE']

for i, sido in enumerate(sido_list):
    for j, std in enumerate(std_list):      
        
        df_train1 = Train_data_en[(Train_data_en['CARD_SIDO_NM'] == sido) & (Train_data_en['STD_CLSS_NM'] == std)]                
        df_pred   = Pred_data_en[(Pred_data_en['CARD_SIDO_NM'] == sido) & (Pred_data_en['STD_CLSS_NM'] == std)]
        print("★ ", i, j)
        
        #DF가 비어있을때 학습X
        if( df_train1.empty): 
            print("train가 비어있음")
            continue

        y_train = df_train1['AMT'].to_numpy(dtype=np.float32).reshape(-1,1)
        X_train = df_train1.drop(['AMT','CARD_SIDO_NM','STD_CLSS_NM'],axis=1).to_numpy(dtype=np.float32)
        X_test = df_pred.drop(['CARD_SIDO_NM','STD_CLSS_NM'],axis=1).to_numpy(dtype=np.float32)        

        print("sido: {}번째 / std: {}번째".format(sido, std))
        print('TRAIN : ',X_train.shape,y_train.shape,
             '\nTEST : ',X_test.shape)

        print('-'*30)

        num_hidden_units = 150
        input_dim = X_train.shape[1]

        model = Sequential()
        model.add(Dense(num_hidden_units,input_dim = input_dim, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(num_hidden_units,kernel_initializer='normal', activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(num_hidden_units,kernel_initializer='normal', activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(num_hidden_units,kernel_initializer='normal', activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(num_hidden_units, kernel_initializer='normal',activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(num_hidden_units,kernel_initializer='normal', activation='relu'))
        model.add(Dense(1))
        model.compile(loss='MeanAbsolutePercentageError', optimizer='adam', metrics=['accuracy'])
        model.fit(X_train, y_train, epochs=500, batch_size=3, verbose=1)
        
        # 예측
        pred = model.predict(X_test)
        df_pred['AMT'] = np.round(pred, 0)
        df_pred['REG_YYMM'] = ('20200' + df_pred['month']).astype('int64')
#         pred_data.drop(['month'],axis=1,inplace=True)
        df_pred['CARD_SIDO_NM'] = sido
        df_pred['STD_CLSS_NM']  = std
        df_pred = df_pred[['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']]
        df_pred = df_pred.groupby(['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM']).sum().reset_index(drop=False) 
        fin = pd.DataFrame(df_pred[0:1])
        final_data.append(fin.values)
        fin = pd.DataFrame(df_pred[1:2])
        final_data.append(fin.values)

★  0 0
sido: 경기번째 / std: 건강보조식품 소매업번째
TRAIN :  (1929, 240) (1929, 1) 
TEST :  (3528, 240)
------------------------------
Epoch 1/500
643/643 [==============================] - 1s 1ms/step - loss: 92.4233 - accuracy: 0.0000e+00
Epoch 2/500
643/643 [==============================] - 1s 1ms/step - loss: 86.7253 - accuracy: 0.0000e+00
Epoch 3/500
643/643 [==============================] - 1s 1ms/step - loss: 80.9690 - accuracy: 0.0000e+00
Epoch 4/500
643/643 [==============================] - 1s 1ms/step - loss: 78.8265 - accuracy: 0.0000e+00
Epoch 5/500
643/643 [==============================] - 1s 1ms/step - loss: 77.0902 - accuracy: 0.0000e+00
Epoch 6/500
643/643 [==============================] - 1s 1ms/step - loss: 75.1338 - accuracy: 0.0000e+00
Epoch 7/500
643/643 [==============================] - 1s 1ms/step - loss: 74.2809 - accuracy: 0.0000e+00
Epoch 8/500
643/643 [==============================] - 1s 1ms/step - loss: 72.2164 - accuracy: 0.0000e+00
Epoch 9/500
643/643 [==========

643/643 [==============================] - 1s 1ms/step - loss: 33.6312 - accuracy: 0.0000e+00
Epoch 77/500
643/643 [==============================] - 1s 1ms/step - loss: 33.4313 - accuracy: 0.0000e+00
Epoch 78/500
643/643 [==============================] - 1s 1ms/step - loss: 33.4322 - accuracy: 0.0000e+00
Epoch 79/500
643/643 [==============================] - 1s 1ms/step - loss: 32.9341 - accuracy: 0.0000e+00
Epoch 80/500
643/643 [==============================] - 1s 1ms/step - loss: 32.9719 - accuracy: 0.0000e+00
Epoch 81/500
643/643 [==============================] - 1s 1ms/step - loss: 33.0298 - accuracy: 0.0000e+00
Epoch 82/500
643/643 [==============================] - 1s 1ms/step - loss: 33.4998 - accuracy: 0.0000e+00
Epoch 83/500
643/643 [==============================] - 1s 1ms/step - loss: 33.7929 - accuracy: 0.0000e+00
Epoch 84/500
643/643 [==============================] - 1s 1ms/step - loss: 31.8030 - accuracy: 0.0000e+00
Epoch 85/500
643/643 [============================

643/643 [==============================] - 1s 1ms/step - loss: 28.6514 - accuracy: 0.0000e+00
Epoch 152/500
643/643 [==============================] - 1s 1ms/step - loss: 29.0820 - accuracy: 0.0000e+00
Epoch 153/500
643/643 [==============================] - 1s 1ms/step - loss: 28.4856 - accuracy: 0.0000e+00
Epoch 154/500
643/643 [==============================] - 1s 1ms/step - loss: 27.5926 - accuracy: 0.0000e+00
Epoch 155/500
643/643 [==============================] - 1s 1ms/step - loss: 27.7829 - accuracy: 0.0000e+00
Epoch 156/500
643/643 [==============================] - 1s 1ms/step - loss: 28.2926 - accuracy: 0.0000e+00
Epoch 157/500
643/643 [==============================] - 1s 1ms/step - loss: 27.9187 - accuracy: 0.0000e+00A: 0s - loss: 28.1832 - accura
Epoch 158/500
643/643 [==============================] - 1s 1ms/step - loss: 28.7682 - accuracy: 0.0000e+00
Epoch 159/500
643/643 [==============================] - 1s 1ms/step - loss: 28.3167 - accuracy: 0.0000e+00
Epoch 160/50

Epoch 226/500
643/643 [==============================] - 1s 1ms/step - loss: 25.9286 - accuracy: 0.0000e+00A: 0s - loss: 24.7558 - accura
Epoch 227/500
643/643 [==============================] - 1s 1ms/step - loss: 25.6789 - accuracy: 0.0000e+00
Epoch 228/500
643/643 [==============================] - 1s 1ms/step - loss: 26.6480 - accuracy: 0.0000e+00
Epoch 229/500
643/643 [==============================] - 1s 1ms/step - loss: 26.2982 - accuracy: 0.0000e+00
Epoch 230/500
643/643 [==============================] - 1s 1ms/step - loss: 25.8529 - accuracy: 0.0000e+00
Epoch 231/500
643/643 [==============================] - 1s 1ms/step - loss: 26.3296 - accuracy: 0.0000e+00
Epoch 232/500
643/643 [==============================] - 1s 1ms/step - loss: 26.0404 - accuracy: 0.0000e+00
Epoch 233/500
643/643 [==============================] - 1s 1ms/step - loss: 26.2755 - accuracy: 0.0000e+00
Epoch 234/500
643/643 [==============================] - 1s 1ms/step - loss: 25.4002 - accuracy: 0.0000e+0

643/643 [==============================] - 1s 1ms/step - loss: 24.3617 - accuracy: 0.0000e+00
Epoch 302/500
643/643 [==============================] - 1s 1ms/step - loss: 24.3099 - accuracy: 0.0000e+00
Epoch 303/500
643/643 [==============================] - 1s 1ms/step - loss: 24.5571 - accuracy: 0.0000e+00
Epoch 304/500
643/643 [==============================] - 1s 1ms/step - loss: 24.7906 - accuracy: 0.0000e+00
Epoch 305/500
643/643 [==============================] - 1s 1ms/step - loss: 24.4161 - accuracy: 0.0000e+00
Epoch 306/500
643/643 [==============================] - 1s 1ms/step - loss: 24.6041 - accuracy: 0.0000e+00
Epoch 307/500
643/643 [==============================] - 1s 1ms/step - loss: 24.1688 - accuracy: 0.0000e+00
Epoch 308/500
643/643 [==============================] - 1s 1ms/step - loss: 23.8769 - accuracy: 0.0000e+00
Epoch 309/500
643/643 [==============================] - 1s 1ms/step - loss: 24.8941 - accuracy: 0.0000e+00
Epoch 310/500
643/643 [===================

...

93/93 [==============================] - 0s 1ms/step - loss: 19.7802 - accuracy: 0.0000e+00
Epoch 322/500
93/93 [==============================] - 0s 1ms/step - loss: 20.4903 - accuracy: 0.0000e+00
Epoch 323/500
93/93 [==============================] - 0s 1ms/step - loss: 21.2429 - accuracy: 0.0000e+00
Epoch 324/500
93/93 [==============================] - 0s 1ms/step - loss: 22.1829 - accuracy: 0.0000e+00
Epoch 325/500
93/93 [==============================] - 0s 1ms/step - loss: 20.2318 - accuracy: 0.0000e+00
Epoch 326/500
93/93 [==============================] - 0s 1ms/step - loss: 20.6974 - accuracy: 0.0000e+00
Epoch 327/500
93/93 [==============================] - 0s 1ms/step - loss: 20.5780 - accuracy: 0.0000e+00
Epoch 328/500
93/93 [==============================] - 0s 1ms/step - loss: 20.3737 - accuracy: 0.0000e+00
Epoch 329/500
93/93 [==============================] - 0s 1ms/step - loss: 22.5435 - accuracy: 0.0000e+00
Epoch 330/500
93/93 [==============================] - 0s 1m

93/93 [==============================] - 0s 1ms/step - loss: 18.9283 - accuracy: 0.0000e+00
Epoch 398/500
93/93 [==============================] - 0s 1ms/step - loss: 18.9891 - accuracy: 0.0000e+00
Epoch 399/500
93/93 [==============================] - 0s 1ms/step - loss: 20.6240 - accuracy: 0.0000e+00
Epoch 400/500
93/93 [==============================] - 0s 1ms/step - loss: 19.9087 - accuracy: 0.0000e+00
Epoch 401/500
93/93 [==============================] - 0s 1ms/step - loss: 19.0291 - accuracy: 0.0000e+00
Epoch 402/500
93/93 [==============================] - 0s 1ms/step - loss: 17.7484 - accuracy: 0.0000e+00
Epoch 403/500
93/93 [==============================] - 0s 1ms/step - loss: 19.3532 - accuracy: 0.0000e+00
Epoch 404/500
93/93 [==============================] - 0s 1ms/step - loss: 21.3291 - accuracy: 0.0000e+00
Epoch 405/500
93/93 [==============================] - 0s 1ms/step - loss: 20.0328 - accuracy: 0.0000e+00
Epoch 406/500
93/93 [==============================] - 0s 1m

93/93 [==============================] - 0s 1ms/step - loss: 18.5159 - accuracy: 0.0000e+00
Epoch 475/500
93/93 [==============================] - 0s 1ms/step - loss: 18.3857 - accuracy: 0.0000e+00
Epoch 476/500
93/93 [==============================] - 0s 1ms/step - loss: 17.8142 - accuracy: 0.0000e+00
Epoch 477/500
93/93 [==============================] - 0s 1ms/step - loss: 17.5343 - accuracy: 0.0000e+00
Epoch 478/500
93/93 [==============================] - 0s 1ms/step - loss: 18.9401 - accuracy: 0.0000e+00
Epoch 479/500
93/93 [==============================] - 0s 1ms/step - loss: 19.2444 - accuracy: 0.0000e+00
Epoch 480/500
93/93 [==============================] - 0s 1ms/step - loss: 18.9915 - accuracy: 0.0000e+00
Epoch 481/500
93/93 [==============================] - 0s 1ms/step - loss: 18.3734 - accuracy: 0.0000e+00A: 0s - loss: 18.3068 - accuracy: 0.0000e+0
Epoch 482/500
93/93 [==============================] - 0s 1ms/step - loss: 18.7047 - accuracy: 0.0000e+00
Epoch 483/500
93/

558/558 [==============================] - 1s 1ms/step - loss: 41.4576 - accuracy: 0.0000e+00
Epoch 50/500
558/558 [==============================] - 1s 1ms/step - loss: 41.1735 - accuracy: 0.0000e+00
Epoch 51/500
558/558 [==============================] - 1s 1ms/step - loss: 39.7977 - accuracy: 0.0000e+00
Epoch 52/500
558/558 [==============================] - 1s 1ms/step - loss: 39.4288 - accuracy: 0.0000e+00
Epoch 53/500
558/558 [==============================] - 1s 1ms/step - loss: 39.4666 - accuracy: 0.0000e+00
Epoch 54/500
558/558 [==============================] - 1s 1ms/step - loss: 38.9547 - accuracy: 0.0000e+00
Epoch 55/500
558/558 [==============================] - 1s 1ms/step - loss: 38.1142 - accuracy: 0.0000e+00
Epoch 56/500
558/558 [==============================] - 1s 1ms/step - loss: 38.8803 - accuracy: 0.0000e+00
Epoch 57/500
558/558 [==============================] - 1s 1ms/step - loss: 36.7903 - accuracy: 0.0000e+00
Epoch 58/500
558/558 [============================

558/558 [==============================] - 1s 1ms/step - loss: 29.7237 - accuracy: 0.0000e+00
Epoch 126/500
558/558 [==============================] - 1s 1ms/step - loss: 28.7977 - accuracy: 0.0000e+00
Epoch 127/500
558/558 [==============================] - 1s 1ms/step - loss: 29.1549 - accuracy: 0.0000e+00
Epoch 128/500
558/558 [==============================] - 1s 1ms/step - loss: 29.1786 - accuracy: 0.0000e+00
Epoch 129/500
558/558 [==============================] - 1s 1ms/step - loss: 29.9995 - accuracy: 0.0000e+00
Epoch 130/500
558/558 [==============================] - 1s 1ms/step - loss: 29.1227 - accuracy: 0.0000e+00
Epoch 131/500
558/558 [==============================] - 1s 1ms/step - loss: 29.4347 - accuracy: 0.0000e+00
Epoch 132/500
558/558 [==============================] - 1s 1ms/step - loss: 29.1563 - accuracy: 0.0000e+00
Epoch 133/500
558/558 [==============================] - 1s 1ms/step - loss: 29.1295 - accuracy: 0.0000e+00
Epoch 134/500
558/558 [===================

### Pred_DF with AMT

In [ ]:
col = ['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM', 'AMT']
Final_data = pd.DataFrame(columns=col)
for data in range(len(final_data)):
    frist_df = pd.DataFrame(final_data[data],columns=col)
    Final_data = Final_data.append(frist_df)
Final_data

### Save to csv

In [ ]:
submission = pd.read_csv('submission_mingsu.csv',index_col = 0)
submission = submission.drop(['AMT'], axis=1)
submission = submission.merge(Final_data, left_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], right_on=['REG_YYMM', 'CARD_SIDO_NM', 'STD_CLSS_NM'], how='left')
submission.index.name = 'id'
submission.to_csv('submission_mingsu(1번째).csv', encoding='utf-8-sig')
submission.head(10)